<a href="https://colab.research.google.com/github/hluling/ph-dash-dev/blob/master/ph_dash_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook setup and Github setup
2022/04/22



In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %cd /content/drive/MyDrive/Github
# !git init ph-dash-dev
# %cd ph-dash-dev
# !git remote add origin https://github.com/hluling/ph-dash-dev
#!git add .
# !git commit -m 'repo setup'
# !git config --global user.email "hluling@outlook.com"
# !git push -u origin master

# Install Libraries

In [1]:
import requests
import pandas as pd
from io import StringIO

## Query API

In [ ]:
response = requests.get("https://api.gdeltproject.org/api/v2/tv/tv?query=%22puerto%20rico%22%20(station:CNN%20OR%20station:FOXNEWS)&mode=timelinevol&format=html&datanorm=perc&timelinesmooth=5&datacomb=sep&last24=yes&timezoom=no&STARTDATETIME=20170829120000&ENDDATETIME=20171007120000")

In [ ]:
response_csv = requests.get("https://api.gdeltproject.org/api/v2/tv/tv?query=%22puerto%20rico%22%20(station:CNN%20OR%20station:FOXNEWS)&mode=timelinevol&format=html&datanorm=perc&timelinesmooth=5&format=csv&datacomb=sep&last24=yes&timezoom=no&STARTDATETIME=20210829120000&ENDDATETIME=20211007120000")
text=StringIO(response_csv.content.decode('utf-8'))

In [ ]:
df=pd.read_csv(text)

In [ ]:
df

,Date (Daily +00:00: 08/29/2021 - 10/07/2021),Series,Value
0,2021-09-02,CNN,0.0170
1,2021-09-03,CNN,0.0170
2,2021-09-04,CNN,0.0170
3,2021-09-05,CNN,0.0042
4,2021-09-06,CNN,0.0042
...,...,...,...
67,2021-10-03,FOXNEWS,0.0241
68,2021-10-04,FOXNEWS,0.0241
69,2021-10-05,FOXNEWS,0.0241
70,2021-10-06,FOXNEWS,0.0098


## Search API: a quick test

In [ ]:
query_url = "https://api.gdeltproject.org/api/v2/tv/tv?query=(kremlin%20OR%20russia%20OR%20putin)%20(ukraine%20OR%20ukrainian%20OR%20zelenskyy)%20market:%22National%22&mode=timelinevol&format=html&datanorm=perc&format=csv&timelinesmooth=5&datacomb=sep&timezoom=yes&STARTDATETIME=20220224120000&ENDDATETIME=20220630120000"

In [2]:
def to_df(queryurl):
  response = requests.get(queryurl)
  content_text = StringIO(response.content.decode('utf-8'))
  df = pd.read_csv(content_text)
  return df

In [ ]:
test_df = to_df(query_url)

In [ ]:
test_df # each day corresponds to a group of channels
# value is the percentage of number of matched 15-seconds clips over the number of monitored 15-seconds clips (because stations may devote different lengths of time to tv news)
# if want raw count, then set datanorm=raw (now datanorm=perc)

,Date (Daily +00:00: 02/24/2022 - 06/30/2022),Series,Value
0,2022-02-28,BLOOMBERG,4.6970
1,2022-03-01,BLOOMBERG,4.3581
2,2022-03-02,BLOOMBERG,3.7279
3,2022-03-03,BLOOMBERG,4.4902
4,2022-03-04,BLOOMBERG,4.5878
...,...,...,...
1102,2022-06-26,MSNBC,0.1914
1103,2022-06-27,MSNBC,0.2816
1104,2022-06-28,MSNBC,0.3406
1105,2022-06-29,MSNBC,0.4955


In [ ]:
test_df.head(30)

In [ ]:
test_df.Series.unique()

array(['BLOOMBERG', 'CNBC', 'CNN', 'CSPAN', 'CSPAN2', 'CSPAN3', 'FBC',
       'FOXNEWS', 'MSNBC'], dtype=object)

In [86]:
# The meat of the code for the article starts here

## Retrieve and prepare data for dashboard
2022/7/7

In [37]:
query_url_ukr = "https://api.gdeltproject.org/api/v2/tv/tv?query=(ukraine%20OR%20ukrainian%20OR%20zelenskyy%20OR%20kiev)%20market:%22National%22&mode=timelinevol&format=html&datanorm=perc&format=csv&timelinesmooth=5&datacomb=sep&timezoom=yes&STARTDATETIME=20220224120000&ENDDATETIME=20220630120000"

In [38]:
query_url_rus = "https://api.gdeltproject.org/api/v2/tv/tv?query=(kremlin%20OR%20russia%20OR%20putin%20OR%20moscow%20OR%20russian)%20market:%22National%22&mode=timelinevol&format=html&datanorm=perc&format=csv&timelinesmooth=5&datacomb=sep&timezoom=yes&STARTDATETIME=20220224120000&ENDDATETIME=20220630120000"

In [39]:
df_ukr = to_df(query_url_ukr)

In [40]:
df_rus = to_df(query_url_rus)

In [7]:
df_ukr.head()

,Date (Daily +00:00: 02/24/2022 - 06/30/2022),Series,Value
0,2022-02-28,BLOOMBERG,11.4829
1,2022-03-01,BLOOMBERG,11.0797
2,2022-03-02,BLOOMBERG,9.9667
3,2022-03-03,BLOOMBERG,12.4323
4,2022-03-04,BLOOMBERG,13.6899


In [8]:
df_rus.head()

,Date (Daily +00:00: 02/24/2022 - 06/30/2022),Series,Value
0,2022-02-28,BLOOMBERG,22.1849
1,2022-03-01,BLOOMBERG,21.7927
2,2022-03-02,BLOOMBERG,19.2834
3,2022-03-03,BLOOMBERG,23.0547
4,2022-03-04,BLOOMBERG,23.7903


In [41]:
df_ukr = df_ukr.rename(columns={"Date (Daily +00:00: 02/24/2022 - 06/30/2022)": "date_col"})
df_rus = df_rus.rename(columns={"Date (Daily +00:00: 02/24/2022 - 06/30/2022)": "date_col"})

In [ ]:
df_ukr.dtypes

date_col     object
Series       object
Value       float64
dtype: object

In [42]:
df_ukr['date_col'] = pd.to_datetime(df_ukr['date_col'])

In [67]:
df_ukr.dtypes

date_col    datetime64[ns]
Series              object
Value              float64
dtype: object

In [43]:
df_rus['date_col'] = pd.to_datetime(df_rus['date_col'])

In [45]:
df_rus = df_rus[[x in ['CNN', 'FOXNEWS', 'MSNBC'] for x in df_rus.Series]]

In [47]:
df_ukr = df_ukr[[x in ['CNN', 'FOXNEWS', 'MSNBC'] for x in df_ukr.Series]]

# Dashboard part starts here

In [13]:
!pip install jupyter-dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.8 MB 7.0 MB/s 
     |████████████████████████████████| 357 kB 54.2 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=cf5bc6d24e85afd794c2e57748346fa586ca88a7b5ce79d8af0940ce57c8a3c0
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [14]:
!pip install dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
!pip install dash_bootstrap_components

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 6.3 MB/s 


In [16]:
import dash
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px

In [84]:
# Initiate a dashboard instance
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LITERA])
server = app.server

# Dashboard front end
app.layout = dbc.Container(
    [   dbc.Row([
        dbc.Col([html.H1('US National Television News Coverage of the War in Ukrain')],
        #html.Hr()],
        className="text-center mt-3 mb-1")
    ]
    ),
        dbc.Row([
            dbc.Label("Select a date range:", className="fw-bold")
    ]),
    
     dbc.Row([
              dcc.DatePickerRange(
                id='date-range',
                min_date_allowed=df_ukr['date_col'].min().date(),
                max_date_allowed=df_ukr['date_col'].max().date(),
                initial_visible_month=df_ukr['date_col'].min().date(),
                start_date=df_ukr['date_col'].min().date(),
                end_date=df_ukr['date_col'].max().date()
              )
    ]),

     dbc.Row([
              dbc.Col(dcc.Graph(id='line-graph-ukr'), 
                      )
     ]),

    dbc.Row([
              dbc.Col(dcc.Graph(id='line-graph-rus'), 
                      )
     ])

    ])

# Dashboard back end
@app.callback(
    Output('line-graph-ukr', 'figure'),
    Output('line-graph-rus', 'figure'),
    Input('date-range', 'start_date'),
    Input('date-range', 'end_date')   
)
def update_output(start_date, end_date):
    mask_ukr = (df_ukr['date_col'] >= start_date) & (df_ukr['date_col'] <= end_date)
    mask_rus = (df_rus['date_col'] >= start_date) & (df_rus['date_col'] <= end_date)

    df_ukr_filtered = df_ukr.loc[mask_ukr]
    df_rus_filtered = df_rus.loc[mask_rus]
    
    line_fig_ukr = px.line(df_ukr_filtered, x="date_col", y="Value", 
                     color='Series', title="Coverage of Ukranian Keywords")
    line_fig_rus = px.line(df_rus_filtered, x='date_col', y='Value', 
                     color='Series', title="Coverage of Russian Keywords")

    line_fig_ukr.update_layout(
                   xaxis_title='Date',
                   yaxis_title='Percentage of Airtime')
    
    line_fig_rus.update_layout(
                   xaxis_title='Date',
                   yaxis_title='Percentage of Airtime')

    line_fig_ukr.update_xaxes(tickformat="%b %d<br>%Y")
    line_fig_rus.update_xaxes(tickformat="%b %d<br>%Y")
    
    return line_fig_ukr, line_fig_rus

In [85]:
# Show dashboard inside the notebook
app.run_server(debug=True, mode="inline")

<IPython.core.display.Javascript object>

In [ ]:
# Show dashboard in browser (in a separate window)
app.run_server(debug=True)

# Recycle bin:

In [57]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LITERA])
server = app.server

In [65]:
from datetime import date

In [70]:
mask_ukr = (df_ukr['date_col'].dt.date >= date(2022, 3, 1)) & (df_ukr['date_col'].dt.date <= date(2022, 3, 31))
mask_rus = (df_rus['date_col'].dt.date >= date(2022, 3, 1)) & (df_rus['date_col'].dt.date <= date(2022, 3, 31))
df_ukr_filtered = df_ukr.loc[mask_ukr]
df_rus_filtered = df_rus.loc[mask_rus]

print(df_ukr_filtered['date_col'])

247    2022-03-01
248    2022-03-02
249    2022-03-03
250    2022-03-04
251    2022-03-05
          ...    
1011   2022-03-27
1012   2022-03-28
1013   2022-03-29
1014   2022-03-30
1015   2022-03-31
Name: date_col, Length: 93, dtype: datetime64[ns]


In [72]:
fig = px.line(df_ukr_filtered, x="date_col", y="Value", 
                     color='Series', title="UKR Results")

In [75]:
fig.update_xaxes(
    #dtick="M1",
    tickformat="%b %d<br>%Y")
fig.show()

In [78]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LITERA])
server = app.server

app.layout = dbc.Container(
    [   dbc.Row([
        dbc.Col([html.H1('News Coverage TV Keyword')],
        #html.Hr()],
        className="text-center mt-3 mb-1")
    ]
    ),
        dbc.Row([

        dbc.Col([
            dbc.Label("Drag to modify date range:", className="fw-bold"),
            dcc.DatePickerRange(
                id='date-range',
                min_date_allowed=df_ukr['date_col'].min().date(),
                max_date_allowed=df_ukr['date_col'].max().date(),
                initial_visible_month=df_ukr['date_col'].min().date(),
                start_date=df_ukr['date_col'].min().date(),
                end_date=df_ukr['date_col'].max().date()
              )
             ],
        #width={'size': 8, "offset": 1}
        )
    ]),
    #  dbc.Row([
    #           dbc.Col(dcc.Graph(id='graph-ukr'), 
    #                   width={'size': 6, 'offset': 0}),
    #           dbc.Col(dcc.Graph(id='graph-rus'), 
    #                   width={'size': 6, 'offset': 0})
    #  ]),
    #  dbc.Row([
    #           dbc.Col(dcc.Graph(id='line-graph-ukr'), 
    #                   width={'size': 6, 'offset': 0}),
    #           dbc.Col(dcc.Graph(id='line-graph-rus'), 
    #                   width={'size': 6, 'offset': 0})
    #  ])
     dbc.Row([
              dbc.Col(dcc.Graph(id='line-graph-ukr'), 
                      )
     ]),

    dbc.Row([
              dbc.Col(dcc.Graph(id='line-graph-rus'), 
                      )
     ])

    ])

@app.callback(
    # Output('graph-ukr', 'figure'),
    # Output('graph-rus', 'figure'),
    Output('line-graph-ukr', 'figure'),
    Output('line-graph-rus', 'figure'),
    Input('date-range', 'start_date'),
    Input('date-range', 'end_date')   
)
def update_output(start_date, end_date):
    mask_ukr = (df_ukr['date_col'] >= start_date) & (df_ukr['date_col'] <= end_date)
    mask_rus = (df_rus['date_col'] >= start_date) & (df_rus['date_col'] <= end_date)

    df_ukr_filtered = df_ukr.loc[mask_ukr]
    df_rus_filtered = df_rus.loc[mask_rus]

    print(df_ukr_filtered['Series'])

    #fig_ukr = px.bar(df_ukr_filtered, x="Series", y="Value", title="UKR Results")
    #fig_rus = px.bar(df_rus_filtered, x="Series", y="Value", title="RUS Results")

    # fig_ukr = px.bar(df_ukr_filtered.groupby(['Series']).mean().reset_index(), 
    #                  x="Series", y="Value", title="UKR Results")
    # fig_rus = px.bar(df_rus_filtered.groupby(['Series']).mean().reset_index(), 
    #                  x="Series", y="Value", title="RUS Results")
    
    line_fig_ukr = px.line(df_ukr_filtered, x="date_col", y="Value", 
                     color='Series', title="Coverage of Ukranian Keywords")
    line_fig_rus = px.line(df_rus_filtered, x='date_col', y='Value', 
                     color='Series', title="Coverage of Russian Keywords")

    # line_fig_ukr.update_layout(
    #     xaxis=dict(
    #         # showline=True,
    #         # showgrid=False,
    #         # showticklabels=True,
    #         # linecolor='rgb(204, 204, 204)',
    #         # linewidth=2,
    #         # ticks='outside',
    #         tickfont=dict(
    #             #family='Arial',
    #             size=8,
    #             #color='rgb(82, 82, 82)',
    #         ),
    #     )
    # )

    line_fig_ukr.update_layout(
                   xaxis_title='Date',
                   yaxis_title='Percentage of Airtime')
    
    line_fig_rus.update_layout(
                   xaxis_title='Date',
                   yaxis_title='Percentage of Airtime')

    line_fig_ukr.update_xaxes(tickformat="%b %d<br>%Y")
    line_fig_rus.update_xaxes(tickformat="%b %d<br>%Y")
    #return fig_ukr, fig_rus, line_fig_ukr, line_fig_rus
    return line_fig_ukr, line_fig_rus

In [ ]:
app.run_server(debug=True, mode="inline")